<a href="https://colab.research.google.com/github/j0el/stock/blob/main/TradingSessions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vectorbt
!pip install yfinance

     |████████████████████████████████| 489 kB 4.3 MB/s 
     |████████████████████████████████| 23.9 MB 11 kB/s 
     |████████████████████████████████| 279 kB 69.8 MB/s 
     |████████████████████████████████| 79.9 MB 48 kB/s 
     |████████████████████████████████| 3.4 MB 28.4 MB/s 
     |████████████████████████████████| 25.3 MB 77 kB/s 
  Created wheel for vectorbt: filename=vectorbt-0.21.0-py3-none-any.whl size=535734 sha256=84f6f63d97722c3cdc70246fe142f54d66cf181d070f26ea7e07fea37834bc06
  Stored in directory: /root/.cache/pip/wheels/0b/eb/c8/6965d7bab40d2e7ccbdb2ccb18cab13723486103e831d1c395
Successfully built vectorbt
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
  Attempting uninstall: numba
    Found existing i

In [2]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import timedelta

In [3]:
# Generate sample price
price_idx = pd.date_range('2018-01-01 12:00:00', periods=48, freq='H')
np.random.seed(42)
price = pd.Series(np.random.uniform(size=price_idx.shape), index=price_idx)
print(price)

2018-01-01 12:00:00    0.374540
2018-01-01 13:00:00    0.950714
2018-01-01 14:00:00    0.731994
2018-01-01 15:00:00    0.598658
2018-01-01 16:00:00    0.156019
2018-01-01 17:00:00    0.155995
2018-01-01 18:00:00    0.058084
2018-01-01 19:00:00    0.866176
2018-01-01 20:00:00    0.601115
2018-01-01 21:00:00    0.708073
2018-01-01 22:00:00    0.020584
2018-01-01 23:00:00    0.969910
2018-01-02 00:00:00    0.832443
2018-01-02 01:00:00    0.212339
2018-01-02 02:00:00    0.181825
2018-01-02 03:00:00    0.183405
2018-01-02 04:00:00    0.304242
2018-01-02 05:00:00    0.524756
2018-01-02 06:00:00    0.431945
2018-01-02 07:00:00    0.291229
2018-01-02 08:00:00    0.611853
2018-01-02 09:00:00    0.139494
2018-01-02 10:00:00    0.292145
2018-01-02 11:00:00    0.366362
2018-01-02 12:00:00    0.456070
2018-01-02 13:00:00    0.785176
2018-01-02 14:00:00    0.199674
2018-01-02 15:00:00    0.514234
2018-01-02 16:00:00    0.592415
2018-01-02 17:00:00    0.046450
2018-01-02 18:00:00    0.607545
2018-01-

In [4]:
# Sessions must be equal - fill missing dates
# Fill on first date before 12:00 and on last date after 11:00
first_date = price.index[0].date()
last_date = price.index[-1].date()+timedelta(days=1)
filled_idx = pd.date_range(first_date, last_date, freq='H')
filled_price = price.reindex(filled_idx)
print(filled_price)

2018-01-01 00:00:00   NaN
2018-01-01 01:00:00   NaN
2018-01-01 02:00:00   NaN
2018-01-01 03:00:00   NaN
2018-01-01 04:00:00   NaN
                       ..
2018-01-03 20:00:00   NaN
2018-01-03 21:00:00   NaN
2018-01-03 22:00:00   NaN
2018-01-03 23:00:00   NaN
2018-01-04 00:00:00   NaN
Freq: H, Length: 73, dtype: float64


In [5]:
# Remove dates that are outside of trading sessions
session_price_idx = filled_price.between_time('9:00', '17:00', include_end=False).index
session_price = filled_price.loc[session_price_idx]
print(session_price)

2018-01-01 09:00:00         NaN
2018-01-01 10:00:00         NaN
2018-01-01 11:00:00         NaN
2018-01-01 12:00:00    0.374540
2018-01-01 13:00:00    0.950714
2018-01-01 14:00:00    0.731994
2018-01-01 15:00:00    0.598658
2018-01-01 16:00:00    0.156019
2018-01-02 09:00:00    0.139494
2018-01-02 10:00:00    0.292145
2018-01-02 11:00:00    0.366362
2018-01-02 12:00:00    0.456070
2018-01-02 13:00:00    0.785176
2018-01-02 14:00:00    0.199674
2018-01-02 15:00:00    0.514234
2018-01-02 16:00:00    0.592415
2018-01-03 09:00:00    0.662522
2018-01-03 10:00:00    0.311711
2018-01-03 11:00:00    0.520068
2018-01-03 12:00:00         NaN
2018-01-03 13:00:00         NaN
2018-01-03 14:00:00         NaN
2018-01-03 15:00:00         NaN
2018-01-03 16:00:00         NaN
dtype: float64


In [6]:
# Select first and last ticks of each trading session and split price into ranges between those ticks
start_idxs = session_price.index[session_price.index.hour == 9]
end_idxs = session_price.index[session_price.index.hour == 16]
price_per_session, _ = session_price.vbt(freq='1H').range_split(start_idxs=start_idxs, end_idxs=end_idxs)
print(price_per_session)

split_idx         0         1         2
0               NaN  0.139494  0.662522
1               NaN  0.292145  0.311711
2               NaN  0.366362  0.520068
3          0.374540  0.456070       NaN
4          0.950714  0.785176       NaN
5          0.731994  0.199674       NaN
6          0.598658  0.514234       NaN
7          0.156019  0.592415       NaN


In [7]:
# Run your strategy (here using random signals)
entries, exits = pd.DataFrame.vbt.signals.generate_random_both(price_per_session.shape, n=2, seed=42)
pf = vbt.Portfolio.from_signals(price_per_session, entries, exits, freq='1H')
print(pf.total_return())

0   -0.786858
1    1.466807
2   -0.529509
Name: total_return, dtype: float64
